<a href="https://colab.research.google.com/github/tkchiang/Educational/blob/main/DLS_Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, HBox, VBox, interactive_output
import math

# **Dynamic Light Scattering**

In a typical DLS experiment, the intensity of scattered light is measured over some period of time, and **fluctuations** about the average value are measured and characterized. This characterization is done by computing and analyzing the intensity **auto-correlation**, which describes how similar are two intensity measurements that are separated by some time, $\tau$:

\begin{align}
ACF(\tau) = \frac{\langle I(t) I(t + \tau) \rangle_{t}}{\langle I(t)^2 \rangle_{t}}
\end{align}

The angle brackets $\langle \rangle_{t}$ denote the expected value of whatever is inside, computed by taking the average over all time $t$. This auto-correlation function takes the form of a weighted sum of decaying exponentials:
\begin{align}
ACF(\tau) &\sim \sum_{i=1}^{N} c_i e^{-\Gamma_i \tau} \\
&= c_1 e^{-\Gamma_1 \tau} + c_2 e^{-\Gamma_2 \tau} \,+ \,...\, c_N e^{-\Gamma_N \tau}
\end{align}

where $\Gamma_i$ is the decay rate from a single particle size, and $c_i$ is a measure of the contribution of that component. For a homogeneous sample containing only particles of a single size, the $ACF$ is a single exponential:

\begin{align}
ACF(\tau) &= \exp(-\Gamma \tau)
\end{align}

Notice that at zero lag-time ($\tau = 0$), the auto-correlation is equal to 1, and at infinite lag-time $\tau = \infty$, the auto-correlation approaches 0. In other words, two measurements separated in time by 0 seconds are extremely similar ($=1$), and two measurements taken far apart in time are extremely dissimilar ($=0$).

### **The intensity auto-correlation decay rate depends particle size**

The decay rate $\Gamma$ is directly related to the particle's diffusion coefficient:

\begin{align}
\Gamma = Dq^2
\end{align}

where $q$ is a constant factor determined by the DLS instrument, and is a function of the light source and detection angle:

\begin{align}
q = \frac{4\pi}{\lambda}\sin\left(\frac{\theta}{2}\right)
\end{align}

The diffusion coefficient $D$ is related to the hydrodynamic diameter $d_h$, i.e. particle size, via the Stokes-Einstein equation:

\begin{align}
D = \frac{k_B T}{3 \pi \eta d_h}
\end{align}

In summary, in a DLS experiment, the rate at which the intensity auto-correlations decay is proportional to the speed at which diffusion occurs; smaller particles diffuse faster than larger ones, and the faster they diffuse, the faster the fluctuations in intensity.

In [ ]:
#@title This block of code defines some utility functions.

def ParticleSizeToScatteringAmplitude(particleDiameter):
  # This function converts a particle size to a scattering amplitude
  Amplitude = particleDiameter**6 # Rayleigh scattering: scattering amplitude scales as size^6
  return Amplitude

def ParticleSizeToDiffusionCoefficient(particleDiameter):
  # This function takes a particle size and calculates its diffusion coefficient in water
  kB = 1.38e-23 # Boltzmann constant (Joules/Kelvin)
  T = 298.15 # Temperature (Kelvin)
  eta = 1e-3 # Viscosity of water (Pascal/second)
  DiffusionCoefficient = 1e21*kB*T/(3*np.pi*eta*particleDiameter)
  return DiffusionCoefficient

def getDecayRate(particleDiameter):
  # This function takes a particle size and calculates the decay rate of the DLS autocorrelogram
  q = 0.0358 # Q-value of the Malvern DLS instrument
  D = ParticleSizeToDiffusionCoefficient(particleDiameter)
  Gamma = D*q**2 # Correlogram decay rate
  return Gamma

In [ ]:
#@title This block of code defines the models for the DLS auto-correlogram of monodisperse and bidisperse samples.

def CorrelogramModel(tau_, particleDiameter):
  # This function simulates the DLS autocorrelogram of a monodisperse particle.
  Amplitude = ParticleSizeToScatteringAmplitude(particleDiameter)
  DecayRate = getDecayRate(particleDiameter)
  model = Amplitude * np.exp(-tau_ * DecayRate)
  return model

def CorrelogramModel_Bimodal(tau_, particleDiameter1, particleDiameter2, contaminantPPM):
  # This function simulates the DLS autocorrelogram of a mixture of two particles.
  Concentration1 = 1e6 - contaminantPPM
  Concentration2 = contaminantPPM
  model = Concentration1*CorrelogramModel(tau_, particleDiameter1) + Concentration2*CorrelogramModel(tau_, particleDiameter2)
  model = model/np.max(model)
  return model

In [ ]:
#@title This block of code defines the models for the DLS auto-correlogram of monodisperse and bidisperse samples.

def CorrelogramModel(tau_, particleDiameter):
  # This function simulates the DLS autocorrelogram of a monodisperse particle.
  Amplitude = ParticleSizeToScatteringAmplitude(particleDiameter)
  DecayRate = getDecayRate(particleDiameter)
  model = Amplitude * np.exp(-tau_ * DecayRate)
  return model

def CorrelogramModel_Bimodal(tau_, particleDiameter1, particleDiameter2, contaminantPPM):
  # This function simulates the DLS autocorrelogram of a mixture of two particles.
  Concentration1 = 1e6 - contaminantPPM
  Concentration2 = contaminantPPM
  model = Concentration1*CorrelogramModel(tau_, particleDiameter1) + Concentration2*CorrelogramModel(tau_, particleDiameter2)
  model = model/np.max(model)
  return model

In [ ]:
#@title True particle size distribution, simulated DLS autocorrelogram, and simulated measured particle size distribution:

def plotSimulation(size1, size2, contaminantPPM, noiselevel):
    tau_range = np.logspace(-1, 6, 50)
    C1 = CorrelogramModel_Bimodal(tau_range, size1, size2, 0)  # Pure sample
    C2 = CorrelogramModel_Bimodal(tau_range, size1, size2, 1e6)  # Pure contaminant
    C3 = CorrelogramModel_Bimodal(tau_range, size1, size2, contaminantPPM)  # Contaminated sample
    C3 = C3 + np.random.normal(0, noiselevel/100, len(tau_range)) # Simulated experimental noise

    particle_sizes = np.array([size1, size2])

    # True particle size distribution by number
    number_distribution = np.array([(1e6 - contaminantPPM), contaminantPPM])
    number_distribution = number_distribution/np.sum(number_distribution)

    # Measured particle size distribution by intensity
    scattering_amplitudes = ParticleSizeToScatteringAmplitude(particle_sizes)
    scattering_distribution = scattering_amplitudes*number_distribution
    scattering_distribution = scattering_distribution/np.sum(scattering_distribution)


    fig, axs = plt.subplots(1, 3, figsize=(18, 4))

    axs[0].bar(particle_sizes, number_distribution, color='b', width=10, align='center')
    axs[0].set_xlabel('Particle Size (nm)')
    axs[0].set_ylabel('Distribution')
    axs[0].set_title('TRUE Particle Size Distribution (By Number)')
    axs[0].grid(True)
    axs[0].set_yscale('log')
    axs[0].set_ylim([1e-6, 2])
    axs[0].set_xlim([0, 500])

    axs[1].plot(tau_range, C1, '--', color='b', label='Pure sample', linewidth=2, markersize=12)
    axs[1].plot(tau_range, C2, '--', color='g', label='Pure aggregate', linewidth=2, markersize=12)
    axs[1].plot(tau_range, C3, '-', color='r', label='Sample + Aggregate mixture')
    axs[1].set_xlabel(r'Lag time $\tau$ (microseconds)')
    axs[1].set_ylabel('Intensity Autocorrelation')
    axs[1].set_title('Simulated DLS correlogram')
    axs[1].legend()
    axs[1].grid(True)
    axs[1].set_xlim([1e-1, 1e6])
    axs[1].set_xscale('log')

    axs[2].bar(particle_sizes, scattering_distribution, color='b', width=10, align='center')
    axs[2].set_xlabel('Particle Size (nm)')
    axs[2].set_ylabel('Distribution')
    axs[2].set_title('MEASURED WITH NO NOISE Particle Size Distribution (By Intensity)')
    axs[2].grid(True)
    axs[2].set_yscale('log')
    axs[2].set_ylim([1e-6, 2])
    axs[2].set_xlim([0, 500])

    plt.tight_layout()
    plt.show()

size1_slider = FloatSlider(value=40, min=1, max=500, step=0.01, description='Particle size',
                            layout={'width': '800px'}, style={'description_width': '400px'})
size2_slider = FloatSlider(value=80, min=1, max=500, step=0.1, description='Aggregate size',
                            layout={'width': '800px'}, style={'description_width': '400px'})
ppm_slider = FloatSlider(value=0, min=0, max=1e6, step=0.001, description='Contamination level (ppm)',
                            layout={'width': '800px'}, style={'description_width': '400px'})
noise_slider = FloatSlider(value=0, min=0, max=100, step=0.01, description='Noise level (%)',
                            layout={'width': '800px'}, style={'description_width': '400px'})

output = interactive_output(plotSimulation, {
    'size1': size1_slider,
    'size2': size2_slider,
    'contaminantPPM': ppm_slider,
    'noiselevel': noise_slider
})

ui = VBox([size1_slider, size2_slider, ppm_slider, noise_slider])
display(VBox([ui, output]))